In [58]:
%%capture
%pip install llama-index-llms-openai llama-index-embeddings-openai llama-index-embeddings-huggingface llama-index-finetuning  llama-index-vector-stores-chroma
%pip install python-dotenv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
%%capture
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [12]:
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.llms.anthropic import Anthropic
from llama_index.finetuning import SentenceTransformersFinetuneEngine
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from torch import Tensor
from transformers import AutoModel, AutoTokenizer
from typing import List, Tuple
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
import chromadb
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
chroma_client = chromadb.Client()

In [3]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [4]:
TRAIN_FILES = ["./data/10k/lyft_2021.pdf"]
VAL_FILES = ["./data/10k/uber_2021.pdf"]

TRAIN_CORPUS_FPATH = "./data/train_corpus.json"
VAL_CORPUS_FPATH = "./data/val_corpus.json"

train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files ['./data/10k/lyft_2021.pdf']
Loaded 238 docs


Parsing nodes: 100%|██████████| 238/238 [00:00<00:00, 1024.06it/s]


Parsed 344 nodes
Loading files ['./data/10k/uber_2021.pdf']
Loaded 307 docs


Parsing nodes: 100%|██████████| 307/307 [00:00<00:00, 1085.37it/s]

Parsed 410 nodes


In [50]:
# Generate dataset
train_dataset = generate_qa_embedding_pairs(
    llm=Anthropic(model="claude-3-haiku-20240307"), nodes=train_nodes
)
val_dataset = generate_qa_embedding_pairs(
    llm=Anthropic(model="claude-3-haiku-20240307"), nodes=val_nodes
)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/344 [00:00<?, ?it/s]


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [5]:
# Load dataset

train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

## Finetune

In [4]:
gte_model_id = "Alibaba-NLP/gte-large-en-v1.5"
bge_model_id = "BAAI/bge-small-en"


finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id=bge_model_id,
    model_output_path="bge-ft",
    val_dataset=val_dataset,
)
finetune_engine.finetune()
embed_model = finetune_engine.get_finetuned_model()

NameError: name 'train_dataset' is not defined

## Evaluation

In [16]:
def create_eval_collection():
  try:
    chroma_client.delete_collection("embed")
    embed_collection = chroma_client.create_collection(name="embed")
  except ValueError:
    embed_collection = chroma_client.create_collection(name="embed")
  return embed_collection

def generate_embeddings(model_path: str, texts: List[str]) -> Tensor:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

    # Tokenize the input texts
    batch_dict = tokenizer(texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = outputs.last_hidden_state[:, 0]
    
    # (Optionally) normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings

def upsert_embeddings_batched(collection, model_path: str, texts: List[TextNode], batch_size: int = 10) -> Tensor:
    for start in tqdm(range(0, len(texts), batch_size)):
        print(f"Processing inputs {start} to {start+batch_size}")
        batch = texts[start:start+batch_size]
        batch_texts = [node.text for node in batch]
        batch_embeddings = generate_embeddings(model_path, batch_texts).tolist()
        batch_ids = [node.id_ for node in batch]
        collection.add(
            documents=batch_texts,
            embeddings=batch_embeddings,
            ids=batch_ids
        )

def query_embeddings(collection, embeddings: List[float], n_results: int) -> List[str]:
    results = collection.query(embeddings, n_results=n_results)
    return results["ids"][0]

In [10]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    print("Starting evaluation")
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    collection = create_eval_collection()
    upsert_embeddings_batched(collection, embed_model, nodes)

    eval_results = []
    print("Starting generation")
    for query_id, query in tqdm(queries.items()):
        # TODO: generate embedding here
        retrieved_ids = query_embeddings(collection, query, top_k)
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [17]:
bge = "BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)
df_bge = pd.DataFrame(bge_val_results)
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

Starting evaluation


  0%|          | 0/41 [00:00<?, ?it/s]

Processing inputs 0 to 10


  0%|          | 0/41 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (893) must match the size of tensor b (512) at non-singleton dimension 1

In [18]:
gte = "Alibaba-NLP/gte-large-en-v1.5"
gte_val_results = evaluate(val_dataset, gte)
df_gte = pd.DataFrame(gte_val_results)
hit_rate_gte = df_gte["is_hit"].mean()
hit_rate_gte


Starting evaluation


  0%|          | 0/41 [00:00<?, ?it/s]

Processing inputs 0 to 10


  2%|▏         | 1/41 [00:30<20:08, 30.21s/it]

Processing inputs 10 to 20


...

Processing inputs 260 to 270


 66%|██████▌   | 27/41 [12:19<05:39, 24.26s/it]

Processing inputs 270 to 280


 66%|██████▌   | 27/41 [12:33<06:30, 27.89s/it]


KeyboardInterrupt: 

In [77]:
gte_val_results = evaluate(val_dataset, embed_model, verbose=True)
df_bge = pd.DataFrame(gte_val_results)
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

Starting evaluation


TypeError: 'list' object cannot be interpreted as an integer